In [4]:
# pydantic==1.10.9

In [1]:
import langchain
import pandas

In [3]:
# import llama 2 from langchain
llama = langchain.llms.LLAMA2()

AttributeError: module 'langchain.llms' has no attribute 'LLAMA2'

In [ ]:
# Create a prompt template using langchain for a query of sunday school lesson teacher
# The prompt is a template for a query of sunday school lesson teacher

def answer(query):
    """
    Function housing prompt template
    """

    # Create a prompt template using langchain for a query of sunday school lesson teacher
    # The prompt is a template for a query of sunday school lesson teacher
    prompt = langchain.PromptTemplate(
        # T = teacher, L = lesson, S = scripture, 

